In [1]:
import datetime
import os
import argparse
import torch

from rlpyt.samplers.collections import TrajInfo
from rlpyt.runners.minibatch_rl import MinibatchRlEval, MinibatchRl
from rlpyt.samplers.serial.sampler import SerialSampler
from rlpyt.utils.logging.context import logger_context

from dreamer_agent import DMCDreamerAgent
from algorithm import Dreamer
from envs.dmc import DeepMindControl
from envs.time_limit import TimeLimit
from envs.action_repeat import ActionRepeat
from envs.normalize_actions import NormalizeActions
from envs.wrapper import make_wapper

2024-01-02 12:37:12.811345: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-02 12:37:13.408279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def build_and_train(
    log_dir,
    game="cartpole_balance",
    run_ID=0,
    cuda_idx=None,
    eval=False,
    save_model="last",
    load_model_path=None,
):
    params = torch.load(load_model_path) if load_model_path else {}
    agent_state_dict = params.get("agent_state_dict")
    optimizer_state_dict = params.get("optimizer_state_dict")
    action_repeat = 2
    factory_method = make_wapper(
        DeepMindControl,
        [ActionRepeat, NormalizeActions, TimeLimit],
        [dict(amount=action_repeat), dict(), dict(duration=1000 / action_repeat)],
    )
    sampler = SerialSampler(
        EnvCls=factory_method,
        TrajInfoCls=TrajInfo,
        env_kwargs=dict(name=game),
        eval_env_kwargs=dict(name=game),
        batch_T=1,
        batch_B=1,
        max_decorrelation_steps=0,
        eval_n_envs=10,
        eval_max_steps=int(10e3),
        eval_max_trajectories=5,
    )
    algo = Dreamer(initial_optim_state_dict=optimizer_state_dict)  # Run with defaults.
    agent = DMCDreamerAgent(
        train_noise=0.3,
        eval_noise=0,
        expl_type="additive_gaussian",
        expl_min=None,
        expl_decay=None,
        initial_model_state_dict=agent_state_dict,
    )
    runner_cls = MinibatchRlEval if eval else MinibatchRl
    runner = runner_cls(
        algo=algo,
        agent=agent,
        sampler=sampler,
        n_steps=50000,
        log_interval_steps=1e3,
        affinity=dict(cuda_idx=cuda_idx),
    )
    config = dict(game=game)
    name = "dreamer_" + game
    with logger_context(
        log_dir,
        run_ID,
        name,
        config,
        snapshot_mode=save_model,
        override_prefix=True,
        use_summary_writer=True,
    ):
        runner.train()


In [3]:
log_dir = os.path.abspath('/home/eddy/Projects/RL_project/logs')

build_and_train(
        log_dir,
        game="humanoid_stand",
        run_ID=0,
        cuda_idx=0,
        eval=False,
        save_model="last",
        load_model_path=False,
    )

2024-01-02 12:37:14.240939  | dreamer_humanoid_stand_0 Runner  master CPU affinity: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11].
2024-01-02 12:37:14.241585  | dreamer_humanoid_stand_0 Runner  master Torch threads: 6.


using seed 8863
 DIMENSIONI DELL INITIAL STATE.STOCH =  torch.Size([1, 30])
2024-01-02 12:37:15.639392  | dreamer_humanoid_stand_0 Sampler decorrelating envs, max steps: 0
2024-01-02 12:37:15.640070  | dreamer_humanoid_stand_0 Serial Sampler initialized.
2024-01-02 12:37:15.640533  | dreamer_humanoid_stand_0 Running 50000 iterations of minibatch RL.
2024-01-02 12:37:16.351956  | dreamer_humanoid_stand_0 Initialized agent model on device: cuda:0.
DAI CHE SIAMO VICINIIIIIIII
size passato a SequenceNStepReturnBuffer = 50000
2024-01-02 12:37:16.374410  | dreamer_humanoid_stand_0 Optimizing over 1000 iterations.


/home/eddy/.local/lib/python3.10/site-packages/torch/optim/adam.py:33: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super().__init__(params, defaults)


 DIMENSIONI DELL INITIAL STATE.STOCH =  torch.Size([1, 30])
2024-01-02 12:37:25.205276  | dreamer_humanoid_stand_0 itr #999 saving snapshot...
2024-01-02 12:37:25.252329  | dreamer_humanoid_stand_0 itr #999 saved
2024-01-02 12:37:25.260678  | -----------------------------  ----------
2024-01-02 12:37:25.261435  | Diagnostics/NewCompletedTrajs     2
2024-01-02 12:37:25.261800  | Diagnostics/StepsInTrajWindow  1000
2024-01-02 12:37:25.262171  | Diagnostics/Iteration           999
2024-01-02 12:37:25.262606  | Diagnostics/CumTime (s)           8.87809
2024-01-02 12:37:25.263070  | Diagnostics/CumSteps           1000
2024-01-02 12:37:25.263414  | Diagnostics/CumCompletedTrajs     2
2024-01-02 12:37:25.264142  | Diagnostics/CumUpdates            0
2024-01-02 12:37:25.264487  | Diagnostics/StepsPerSecond      112.637
2024-01-02 12:37:25.264778  | Diagnostics/UpdatesPerSecond      0
2024-01-02 12:37:25.265219  | Diagnostics/ReplayRatio           0
2024-01-02 12:37:25.265687  | Diagnostics/Cum

Imagination:   0%|          | 0/100 [00:00<?, ?it/s]

...

 DIMENSIONI DELL INITIAL STATE.STOCH =  torch.Size([50, 30])


Imagination:   1%|          | 1/100 [00:01<02:16,  1.38s/it]

 DIMENSIONI DELL INITIAL STATE.STOCH =  torch.Size([50, 30])


Imagination:   1%|          | 1/100 [00:02<03:24,  2.07s/it]


KeyboardInterrupt: 